In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger
import os as os
from google.colab import files

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 2
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

x_train = x_train.reshape(x_train.shape[0], img_height, img_width, channel)
x_test = x_test.reshape(x_test.shape[0], img_height, img_width, channel)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # Normalise data to [0, 1] range
x_test /= 255 # Normalise data to [0, 1] range

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

 94642176/170498071 [===============>..............] - ETA: 38s

170500096/170498071 [==============================] - 78s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
       
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
       
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
   
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12  

input = Input(shape=(img_height, img_width, channel,))
Conv2D_1_1 = Conv2D(num_filter, (1,1), use_bias=False)(input)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False)(Conv2D_1_1)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)
  
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   36          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 30, 30, 12)   1296        conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 30, 30, 12)   48          conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation

In [0]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen.fit(x_train)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# checkpoint
filepath="weights.best.Data.Aug.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
logger = CSVLogger(os.path.join("LogFile_Data_Aug.log"))
callbacks_list = [checkpoint, logger]

In [18]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=1000,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test,y_test),
                    callbacks = callbacks_list,
                    shuffle='True')

Epoch 1/50
1000/1000 [==============================] - 798s 798ms/step - loss: 0.7009 - acc: 0.7543 - val_loss: 1.2614 - val_acc: 0.6617

Epoch 00001: val_acc improved from 0.65000 to 0.66170, saving model to weights.best.Data.Aug.hdf5
Epoch 2/50
  16/1000 [..............................] - ETA: 12:34 - loss: 0.6725 - acc: 0.7728

1000/1000 [==============================] - 799s 799ms/step - loss: 0.5966 - acc: 0.7929 - val_loss: 1.0625 - val_acc: 0.7305

Epoch 00002: val_acc improved from 0.66170 to 0.73050, saving model to weights.best.Data.Aug.hdf5
Epoch 3/50
  54/1000 [>.............................] - ETA: 12:17 - loss: 0.5543 - acc: 0.8106

1000/1000 [==============================] - 800s 800ms/step - loss: 0.5289 - acc: 0.8180 - val_loss: 1.1055 - val_acc: 0.7148

Epoch 00003: val_acc did not improve from 0.73050
Epoch 4/50
  77/1000 [=>............................] - ETA: 12:00 - loss: 0.4880 - acc: 0.8361

1000/1000 [==============================] - 799s 799ms/step - loss: 0.4744 - acc: 0.8371 - val_loss: 0.9065 - val_acc: 0.7662

Epoch 00004: val_acc improved from 0.73050 to 0.76620, saving model to weights.best.Data.Aug.hdf5
Epoch 5/50
  70/1000 [=>............................] - ETA: 12:02 - loss: 0.4505 - acc: 0.8473

1000/1000 [==============================] - 799s 799ms/step - loss: 0.4370 - acc: 0.8495 - val_loss: 0.5898 - val_acc: 0.8198

Epoch 00005: val_acc improved from 0.76620 to 0.81980, saving model to weights.best.Data.Aug.hdf5
Epoch 6/50
  68/1000 [=>............................] - ETA: 12:07 - loss: 0.4315 - acc: 0.8486

...

1000/1000 [==============================] - 795s 795ms/step - loss: 0.1197 - acc: 0.9572 - val_loss: 0.4821 - val_acc: 0.8866

Epoch 00039: val_acc did not improve from 0.91210
Epoch 40/50
  86/1000 [=>............................] - ETA: 11:44 - loss: 0.1231 - acc: 0.9557

1000/1000 [==============================] - 795s 795ms/step - loss: 0.1171 - acc: 0.9582 - val_loss: 0.3899 - val_acc: 0.9054

Epoch 00040: val_acc did not improve from 0.91210
Epoch 41/50
  86/1000 [=>............................] - ETA: 11:44 - loss: 0.1129 - acc: 0.9602

1000/1000 [==============================] - 795s 795ms/step - loss: 0.1147 - acc: 0.9593 - val_loss: 0.3329 - val_acc: 0.9175

Epoch 00041: val_acc improved from 0.91210 to 0.91750, saving model to weights.best.Data.Aug.hdf5
Epoch 42/50
  72/1000 [=>............................] - ETA: 11:57 - loss: 0.1178 - acc: 0.9568

 632/1000 [=================>............] - ETA: 4:44 - loss: 0.1124 - acc: 0.9598

KeyboardInterrupt: ignored

In [0]:
#model.fit(x_train, y_train,
#                    batch_size=batch_size,
#                    epochs=epochs,
#                    verbose=1,
#                    validation_data=(x_test, y_test), shuffle='True')

In [21]:
# Save the trained weights in to .h5 format
model.save_weights('model_weights_data_aug.h5')

files.download('model_weights_data_aug.h5')
files.download('LogFile_Data_Aug.log')
files.download('weights.best.data.aug.hdf5')

FileNotFoundError: ignored

In [22]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 29s 3ms/step
Test loss: 0.4070707778275013
Test accuracy: 0.8985


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')